In [41]:
from collections import deque

with open('day10_input.txt') as f:
    lines = f.read().splitlines()
    
def search(target, buttons):
    # target is a tuple of indices that should be ON
    if not target:
        return 0
    seen = set([tuple()])
    q = deque([tuple()])
    pushes = 0
    while q:
        pushes += 1
        for _ in range(len(q)):
            state = q.popleft()
            for b in buttons:
                new_state = set(state)
                for i in b:
                    if i in new_state:
                        new_state.remove(i)
                    else:
                        new_state.add(i)
                new_state = tuple(sorted(new_state))
                if new_state == target:
                    return pushes
                if new_state not in seen:
                    seen.add(new_state)
                    q.append(new_state)
        print(q, target in seen)

res = 0
for line in lines:
    i1 = line.index(']')
    i2 = line.index('{')
    lights = line[1:i1]
    buttons = line[i1+1:i2].strip().split()
    
    target = tuple(i for i in range(len(lights)) if lights[i] == '#')
    buttons = [eval(b) if isinstance(eval(b), tuple) else (eval(b),) for b in buttons]
    print(lights, target, buttons)
    
    s = search(target, buttons)
    print(s)
    res += s
print(res)

..##.#. (2, 3, 5) [(0, 4), (1, 3, 5), (0, 6), (0, 2, 3, 5), (2, 5, 6), (2, 3, 5), (0, 1, 2, 4), (0, 5)]
1
.######.# (1, 2, 3, 4, 5, 6, 8) [(0, 2, 3, 4, 6, 7), (1, 4, 6), (0, 4, 6), (1, 3, 4, 5, 8), (1, 7, 8), (1, 2, 3, 4, 5, 6, 8), (0, 2, 7, 8)]
1
..#### (2, 3, 4, 5) [(0, 1, 2, 3, 4), (0, 1, 3, 4), (0, 1, 2), (0, 1, 3, 5), (4,)]
deque([(0, 1, 2, 3, 4), (0, 1, 3, 4), (0, 1, 2), (0, 1, 3, 5), (4,)]) False
deque([(2,), (3, 4), (2, 4, 5), (0, 1, 2, 3), (2, 3, 4), (4, 5), (0, 1, 3), (2, 3, 5), (0, 1, 2, 4), (0, 1, 3, 4, 5)]) False
3
.#...#.#.. (1, 5, 7) [(3, 5), (1, 2, 3, 4, 6, 8, 9), (0, 4, 8, 9), (2, 3, 4, 5, 6, 7, 8, 9), (6, 8), (1, 4, 5, 7, 8), (1, 2, 5, 9), (0, 1, 3, 4, 5, 6, 9)]
deque([(3, 5), (1, 2, 3, 4, 6, 8, 9), (0, 4, 8, 9), (2, 3, 4, 5, 6, 7, 8, 9), (6, 8), (1, 4, 5, 7, 8), (1, 2, 5, 9), (0, 1, 3, 4, 5, 6, 9)]) False
2
.### (1, 2, 3) [(0, 2), (0, 1, 3), (1, 2)]
deque([(0, 2), (0, 1, 3), (1, 2)]) False
2
.#...### (1, 5, 6, 7) [(1, 6), (0, 2, 5), (0, 2, 3, 7), (0, 3, 5, 6, 7), (0,

In [ ]:
from collections import deque
import heapq
from functools import cache, lru_cache

with open('day10_input copy.txt') as f:
    lines = f.read().splitlines()
    
def button_to_transformed(button, target, mul):
    res = [0] * len(target)
    for i in button:
        res[i] = mul
        if res[i] > target[i]:
            return False, tuple()
    return True, tuple(res)

def preprocess_buttons(buttons, target):
    transformed = set()
    for b in buttons:
        mul = 1
        r, t = button_to_transformed(b, target, mul)
        while r:
            transformed.add((mul, t))
            mul += 1
            r, t = button_to_transformed(b, target, mul)
    return sorted(transformed, reverse=True)

@lru_cache(None)
def backtrack(state, button_idx, target):
    if state == target:
        return 0
    elif button_idx >= len(buttons):
        return float('inf')
    mul, button = buttons[button_idx]
    new_state_list = [0] * len(target)
    for i in range(len(target)):
        # print(state, button, target)
        new_state_list[i] = state[i] + button[i]
        if new_state_list[i] > target[i]:
            return backtrack(state, button_idx + 1, target)
    new_state = tuple(new_state_list)
    res = min(backtrack(new_state, button_idx, target) + mul, backtrack(state, button_idx + 1, target))
    # if res == float('inf'):
    #     res = backtrack(state, button_idx + 1, target)
    return res

res = 0
for line in lines:
    i1 = line.index(']')
    i2 = line.index('{')
    lights = line[1:i1]
    buttons = line[i1+1:i2].strip().split()
    joltage = tuple(map(int, line[i2+1:-1].strip().split(',')))
    
    buttons = [eval(b) if isinstance(eval(b), tuple) else (eval(b),) for b in buttons]
    buttons = preprocess_buttons(buttons, joltage)
    # print(buttons)
    s = backtrack(tuple(0 for _ in range(len(joltage))), 0, joltage)
    print(s)
    res += s
print(res)

In [28]:
from collections import deque
from functools import lru_cache
import numpy as np
import ast

with open('day10_input copy.txt') as f:
    lines = f.read().splitlines()

def preprocess_buttons(buttons, target_arr):
    """Return a sorted list of (mul, np.array) transformed button vectors.

    For each input button (tuple of indices), we compute the maximum multiplicity
    allowed by the target and produce vectors for mul=1..max_mul.
    """
    transformed = []
    for b in buttons:
        # maximum times we can press this button without exceeding target at any index
        if len(b) == 0:
            continue
        max_mul = min(int(target_arr[i]) for i in b)
        for mul in range(1, max_mul + 1):
            arr = np.zeros_like(target_arr)
            arr[list(b)] = mul
            transformed.append((mul, arr))
    # sort heuristically by mul descending (try larger chunks first)
    transformed.sort(key=lambda x: x[0], reverse=True)
    return transformed


@lru_cache(None)
def backtrack(state_tuple, button_idx, main=False):
    """Backtracking with memoization. state_tuple is a tuple of ints."""
    if state_tuple == target_tuple:
        return 0
    if button_idx >= len(buttons_np):
        print('Missed target:', state_tuple, target_tuple)
        return float('inf')

    state_arr = np.array(state_tuple, dtype=int)

    mul, btn_arr = buttons_np[button_idx]
    # try taking this transformed button (adds btn_arr, costs mul)
    new_state_arr = state_arr + btn_arr
    if np.any(new_state_arr > target_arr):
        # cannot take this transformed button; skip to next
        return backtrack(state_tuple, button_idx + 1)
    new_state_tuple = tuple(new_state_arr.tolist())
    take_cost = backtrack(new_state_tuple, button_idx) + mul
    if button_idx < len(buttons_np) * 3 // 4:
        return take_cost
    skip_cost = backtrack(state_tuple, button_idx + 1)
    return min(take_cost, skip_cost)


res = 0
for line in lines:
    i1 = line.index(']')
    i2 = line.index('{')
    lights = line[1:i1]
    button_tokens = line[i1+1:i2].strip().split()
    target_tuple = tuple(map(int, line[i2+1:-1].strip().split(',')))
    target_arr = np.array(target_tuple, dtype=int)

    # safe parsing of button tokens
    buttons = []
    for b in button_tokens:
        v = ast.literal_eval(b)
        if isinstance(v, int):
            buttons.append((v,))
        else:
            buttons.append(tuple(v))

    buttons_np = preprocess_buttons(buttons, target_arr)
    print(buttons_np)
    print(buttons_np[:len(buttons_np) * 3//4])
    # clear cache since target/buttons change per machine
    backtrack.cache_clear()
    s = backtrack(tuple(0 for _ in range(len(target_tuple))), 0)
    print(s)
    res += s
print(res)

[(54, array([54,  0,  0,  0,  0, 54,  0])), (53, array([53,  0,  0,  0,  0, 53,  0])), (52, array([52,  0,  0,  0,  0, 52,  0])), (51, array([51,  0,  0,  0,  0, 51,  0])), (50, array([50,  0,  0,  0,  0, 50,  0])), (49, array([49,  0,  0,  0,  0, 49,  0])), (48, array([48,  0,  0,  0,  0, 48,  0])), (47, array([47,  0,  0,  0,  0, 47,  0])), (46, array([46,  0,  0,  0,  0, 46,  0])), (45, array([45,  0,  0,  0,  0, 45,  0])), (44, array([44,  0,  0,  0,  0, 44,  0])), (43, array([43,  0,  0,  0,  0, 43,  0])), (42, array([42,  0,  0,  0,  0, 42,  0])), (41, array([41,  0,  0,  0,  0, 41,  0])), (40, array([40,  0,  0,  0,  0, 40,  0])), (39, array([39,  0,  0,  0,  0, 39,  0])), (38, array([38,  0,  0,  0,  0, 38,  0])), (37, array([37,  0,  0,  0,  0, 37,  0])), (36, array([36,  0,  0,  0,  0, 36,  0])), (35, array([35,  0,  0,  0,  0, 35,  0])), (34, array([34,  0,  0,  0,  0, 34,  0])), (33, array([33,  0,  0,  0,  0, 33,  0])), (32, array([32,  0,  0,  0,  0, 32,  0])), (31, array